In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import time
from IPython import display

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
train_images = (train_images - 127.5) / 127.5
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(60000).batch(256)
print(train_images.shape)

(60000, 28, 28, 1)


In [3]:
def create_generator():
    model = tf.keras.Sequential()

    # creating Dense layer with units 7*7*256(batch_size) and input_shape of (100,)
    model.add(layers.Dense(7 * 7 * 256, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((7, 7, 256)))

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))

    return model


def create_discriminator():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[28, 28, 1]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model


cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def D_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss


def G_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)


generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [4]:
noise_dim = 100
num_of_generated_examples = 16
BATCH_SIZE = 256

seed = tf.random.normal([num_of_generated_examples, noise_dim])
generator = create_generator()
discriminator = create_discriminator()

tf.Tensor(
[[ 0.8353238   0.38880274  1.4346193  ... -0.64614606 -1.8763728
   0.18219034]
 [ 0.13435683  0.73771083 -1.3796108  ... -0.8506013  -0.48548466
   0.22784023]
 [ 1.5320183  -0.94690174  0.42568567 ... -0.30983248  0.6371806
  -0.13250297]
 ...
 [ 0.95692956  0.2823434  -0.24484774 ...  1.3948464  -0.30968973
   1.2841231 ]
 [-0.65827537 -0.5550861  -0.6748325  ...  1.4578443  -0.7802931
  -0.3871687 ]
 [-0.94071084 -0.64916456 -0.5961767  ... -0.09056605 -0.560218
  -0.29084933]], shape=(16, 100), dtype=float32)


In [54]:
@tf.function
def train_step(images, arg, epoch):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])
    
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = G_loss(fake_output)
        disc_loss = D_loss(real_output, fake_output)
    
    if arg == 1:
        print("Epoch: {}\nGenerator Loss: {}\nDiscriminator Loss: {}".format(epoch+1, gen_loss, disc_loss))

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [17]:
import os

PROJECT_ROOT_DIR = "."
SAVE_ID = "picture"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, SAVE_ID)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=100):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [25]:
import datetime

log_dir = ".\\log"
run_name = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+"/"
train_file_writer = tf.summary.create_file_writer(log_dir+run_name+"/train")
val_file_writer = tf.summary.create_file_writer(log_dir+run_name+"/validation")

In [53]:
def train_GAN(dataset, epochs):
    for epoch in range(epochs):
        start = time.time()
        count = 0


        for image_batch in dataset:
            count += 1
            if count == 235:
                train_step(image_batch, 1, epoch)
            else:
                train_step(image_batch, 0, epoch)


        print('Time for epoch {} is {} sec'.format(epoch + 1, time.time() - start))
        generated_images = generator(seed, training=False)
        if (epoch + 1) % 15 == 0:
            print('Generated Images after {} epochs'.format(epoch + 1))
            plt.figure(figsize=(10, 10))
            for i in range(generated_images.shape[0]):
                plt.subplot(4, 4, i + 1)
                plt.imshow(generated_images[i, :, :, 0], cmap='gray')
                plt.axis('on')
            save_fig(str(epoch + 1))
            plt.show()


tf.config.run_functions_eagerly(True)
train_GAN(train_dataset, 1200)

Epoch: 0
Generator Loss: 1.054664969444275
Discriminator Loss: 1.2191098928451538
Time for epoch 1 is 20.548791885375977 sec


KeyboardInterrupt: 